In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests as req
import http.cookiejar
import urllib.request
import time as tm
import re
import numpy as np
import json
import datetime

from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from datetime import datetime as dt

pd.set_option('display.max_columns', None)


In [2]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
cookies_path = os.getenv("cookies_path")

sys.path.append(os.getenv('scraper_path'))
from YT_Scrapy import YtScraper
scrapper = YtScraper()

In [3]:
sys.path.append(os.getenv('scraper_path'))
from YT_Scrapy import YtScraper
scrapper = YtScraper()

In [4]:
def load_cookies_from_file(cookie_file):
    cookie_jar = http.cookiejar.MozillaCookieJar(cookie_file)
    cookie_jar.load()
    return cookie_jar

In [5]:
def check_existing_file(path:str):
    return os.path.isfile(path) #check if file is exist 


def save_csv(path:str, df):
    # if file exist, joining existing csv file with new one then save
    if check_existing_file(path): 
        old_df = pd.read_csv(path)
        result = pd.concat([old_df, df], axis=0)
        result.to_csv(path,index=False)
        
    # else save file directly
    else:
        df.to_csv(path,index=False)
        
    del df

In [6]:
def add_rank(df):
    counter = 1
    df['video_rank']=None
    for i in range(df.shape[0]):
        df['video_rank'].iat[i] = counter
        counter +=1
    return df

In [7]:
def video_tracking(latest_data,data_path,file_name,times):
    file_name = f'{data_path}{file_name}_reserve.csv'
    if latest_data is not None:
        if check_existing_file(file_name):
            existing_data = pd.read_csv(file_name)
            existing_data_ids = pd.Series(existing_data['video_id'].unique())#.tolist()
            existing_data_ids = existing_data_ids[~existing_data_ids.isin(latest_data['video_id'])].tolist()
            new_existing_data = scrapper.scrape_video_data(existing_data_ids)
            new_existing_data['time_recorded'] = times
            # latest_data = latest_data.loc[~latest_data['video_id'].isin(new_existing_data['video_id'])]
            new_data = pd.concat([existing_data,new_existing_data,latest_data],axis=0)
            # new_data = new_data.drop_duplicates(subset=['video_id', 'time_recorded'], keep='first')
            new_data.to_csv(file_name,index=False)
            del existing_data,new_existing_data,latest_data,new_data,existing_data_ids
            return None
        else:
            latest_data.to_csv(file_name,index=False)
            del latest_data
            return None
    else:
        if not check_existing_file(file_name):
            return None
        else:
            existing_data = pd.read_csv(file_name)
            existing_data_ids = existing_data['video_id'].unique().tolist()
            new_existing_data = scrapper.scrape_video_data(existing_data_ids)
            new_existing_data['time_recorded'] = times
            new_data = pd.concat([existing_data,new_existing_data],axis=0)
            new_data.to_csv(file_name,index=False)
            del new_data,new_existing_data,existing_data_ids,existing_data
            return None
        
        
def process_and_save(data_type, ids, data_path, file_name, counters, times):
    if len(ids) > 0:
        df = scrapper.scrape_video_data(sorted(ids))
        df['time_recorded'] = times
        video_tracking(df,data_path,file_name,times)
        tm.sleep(10)
        df = add_rank(df)
        save_csv(f'{data_path}{file_name}.csv', df)
        print(f'df {data_type} {counters}')
        tm.sleep(10)
    else:
        video_tracking(None,data_path,file_name,times)

In [8]:
# session = req.Session()

In [ ]:
# counters = 1
# # keywords = ['vtuber','yoasobi','frieren','python','neuro sama','furina','genshin','honor of king']
# for _ in range(100):
#     # for keyword in keywords:
#     cookie_jar = http.cookiejar.LWPCookieJar(cookies_path)
#     session.cookies = cookie_jar
#     times = dt.now()
#     headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'}
#     payload = {
#         'search_query': 'vtuber',
#         'sp':'EgQQAUAB'
#     }
#     html = session.get('https://www.youtube.com/results',params=payload,headers=headers)
#     soup = BeautifulSoup(html.content, 'html.parser')
#     yt_initial_data_match = re.search(r'var ytInitialData = ({.*?});', html.text)
#     if yt_initial_data_match:
#         yt_initial_data = json.loads(yt_initial_data_match.group(1))
#     tmp_data = yt_initial_data.get('contents').get('twoColumnSearchResultsRenderer').get('primaryContents').get('sectionListRenderer').get('contents')
#     for i in range(len(tmp_data)):
#         tmp_data_1 = tmp_data[i].get('itemSectionRenderer').get('contents')
#         if tmp_data_1:
#             tmp_data = tmp_data_1
#             del tmp_data_1
#             break
#         else:
#             pass
#     new_short_id = set()
#     old_short_id = set()
#     video_id = set()
    
#     for i in range(len(tmp_data)):
#         for key in tmp_data[i].keys():
#             if key == 'reelShelfRenderer' and tmp_data[i].get(key).get('title').get('simpleText') == 'Shorts':
#                 if len(tmp_data[i].get(key).get('items')) > 0:
#                     for x in range(len(tmp_data[i].get(key).get('items'))):
#                         data_holder = tmp_data[i].get(key).get('items')[x].get('shortsLockupViewModel').get('onTap').get('innertubeCommand')
#                         old_short_id.add(data_holder.get('reelWatchEndpoint').get('videoId'))
#                         del data_holder
                        
#             elif key == 'reelShelfRenderer' and tmp_data[i].get(key).get('title').get('simpleText') == 'Recently uploaded Shorts':
#                 if len(tmp_data[i].get(key).get('items')) > 0:
#                     for x in range(len(tmp_data[i].get(key).get('items'))):
#                         data_holder = tmp_data[i].get(key).get('items')[x].get('shortsLockupViewModel').get('onTap').get('innertubeCommand')
#                         new_short_id.add(data_holder.get('reelWatchEndpoint').get('videoId'))
#                         del data_holder
                        
#             elif key == 'videoRenderer':
#                 video_id.add(tmp_data[i].get(key).get('videoId'))
#     data_path = 'data/youtube_scraping/'
#     # process_and_save('short old', old_short_id,data_path, 'old_short', counters, times)
    
    
#     # Process new shorts
#     # process_and_save('short new', new_short_id,data_path, 'new_short', counters, times)
    
#     # Process videos
#     process_and_save('video', video_id,data_path, 'video', counters, times)
    
#     # Save cookie and update counters
#     cookie_jar.save(ignore_discard=True, ignore_expires=True)
#     counters += 1
#     tm.sleep(60 * 5)

df video 1
df video 2
df video 3
df video 4
df video 5
df video 6
df video 7
df video 8
df video 9
df video 10
df video 11
df video 12
df video 13
df video 14
df video 15
df video 16
df video 17
df video 18
df video 19
df video 20
df video 21


In [14]:

start_date = datetime.date(2023, 9, 4)
start_date.strftime('%Y-%m-%d')

'2023-09-04'

In [ ]:

start_date = datetime.date(2023, 9, 4)
end_date = datetime.date.today()
end_date -= datetime.timedelta(days=1)
# Inisialisasi loop
current_date = start_date
date_list = []
data_path = 'data/youtube_scraping/'
# Loop dari tanggal start_date hingga end_date
while current_date <= end_date:
    # Format tanggal menjadi string yyyy-mm-dd
    date_str = current_date.strftime('%Y-%m-%d')
    date_list.append(date_str)
    
    # Tambahkan satu hari ke tanggal saat ini
    current_date += datetime.timedelta(days=1)
    tomorow_date = current_date.strftime('%Y-%m-%d')
    df = scrapper.scrape_search_video(
        q='debut stream vtuber',
        regionCode='id',
        publishedAfter=date_str,
        publishedBefore=tomorow_date,
        all_data=True
    )

    save_csv(f'{data_path}vtuber_debut.csv', df)